# SQL : Bike store relational database

In [43]:
import pandas as pd
import sqlite3

In [44]:
# Read in CSV files
brands = pd.read_csv('/data/notebook_files/brands.csv')
categories = pd.read_csv('/data/notebook_files/categories.csv')
customers = pd.read_csv('/data/notebook_files/customers.csv')
order_items = pd.read_csv('/data/notebook_files/order_items.csv')
orders = pd.read_csv('/data/notebook_files/orders.csv')
products = pd.read_csv('/data/notebook_files/products.csv')
staffs = pd.read_csv('/data/notebook_files/staffs.csv')
stocks = pd.read_csv('/data/notebook_files/stocks.csv')
stores = pd.read_csv('/data/notebook_files/stores.csv')

# Create database connection
connection = sqlite3.connect('bike_store.db')

# Insert data into database
brands.to_sql('brands', connection, if_exists='replace', index=False)
categories.to_sql('categories', connection, if_exists='replace', index=False)
customers.to_sql('customers', connection, if_exists='replace', index=False)
order_items.to_sql('order_items', connection, if_exists='replace', index=False)
orders.to_sql('orders', connection, if_exists='replace', index=False)
products.to_sql('products', connection, if_exists='replace', index=False)
staffs.to_sql('staffs', connection, if_exists='replace', index=False)
stocks.to_sql('stocks', connection, if_exists='replace', index=False)
stores.to_sql('stores', connection, if_exists='replace', index=False)

3

**Question 1** What is the total revenue generated from bike sales in the each year?

In [45]:
query = """
SELECT 
    STRFTIME('%Y',od.order_date) year,
    ROUND(SUM(odi.list_price),0) revenue
FROM orders od
JOIN order_items odi
ON od.order_id = odi.order_id
GROUP BY year;
"""

df = pd.read_sql_query(query, connection)
df

,year,revenue
0,2016,1806105.0
1,2017,2567333.0
2,2018,1352969.0


**Question 2** Which bike model is the best seller, and what is its average selling price

In [46]:
# Using GROUP BY
query = """
SELECT
    odi.product_id,
    pd.product_name,
    pd.model_year,
    SUM(odi.quantity) totol_sales,
    ROUND(AVG(odi.list_price),1) average_price
FROM 
    order_items odi, products pd
WHERE
    odi.product_id = pd.product_id
GROUP BY
    1
ORDER BY
    4 DESC;
"""

df = pd.read_sql_query(query, connection)
df

,product_id,product_name,model_year,totol_sales,average_price
0,6,Surly Ice Cream Truck Frameset - 2016,2016,167,470.0
1,13,Electra Cruiser 1 (24-Inch) - 2016,2016,157,270.0
2,16,Electra Townie Original 7D EQ - 2016,2016,156,600.0
3,23,Electra Girl's Hawaii 1 (20-inch) - 2015/2016,2016,154,300.0
4,7,Trek Slash 8 27.5 - 2016,2016,154,4000.0
...,...,...,...,...,...
302,174,Trek Domane SLR Disc Frameset - 2018,2018,1,3200.0
303,170,Trek CrossRip 2 - 2018,2018,1,1300.0
304,167,Surly ECR Frameset - 2018,2018,1,750.0
305,157,Trek Domane SL Frameset Women's - 2018,2018,1,6500.0


In [47]:
# Using Window function
query = """
SELECT
    DISTINCT(pd.product_id),
    pd.product_name,
    pd.model_year,
    SUM(odi.quantity) OVER(PARTITION BY pd.product_id) total_sales,
    ROUND(AVG(odi.list_price) OVER(PARTITION BY pd.product_id),1) average_price
FROM 
    order_items odi, products pd
WHERE
    odi.product_id = pd.product_id
ORDER BY
    4 DESC;
"""

df = pd.read_sql_query(query, connection)
df

,product_id,product_name,model_year,total_sales,average_price
0,6,Surly Ice Cream Truck Frameset - 2016,2016,167,470.0
1,13,Electra Cruiser 1 (24-Inch) - 2016,2016,157,270.0
2,16,Electra Townie Original 7D EQ - 2016,2016,156,600.0
3,7,Trek Slash 8 27.5 - 2016,2016,154,4000.0
4,23,Electra Girl's Hawaii 1 (20-inch) - 2015/2016,2016,154,300.0
...,...,...,...,...,...
302,294,Electra Tiger Shark 3i (20-inch) - Boys' - 2018,2018,1,370.0
303,296,"Electra Treasure 3i 20"" - 2018",2018,1,370.0
304,299,Electra Townie Original 21D - 2018,2018,1,560.0
305,305,Electra Townie Balloon 3i EQ - 2017/2018,2018,1,750.0


**Question 3** What is the average units sold for each month in each year, for each product category

In [48]:
query = """
WITH sub1 AS (
    SELECT
        o.order_id,
        o.order_date,
        od.product_id,
        od.quantity
    FROM 
        orders o, order_items od
    WHERE 
        o.order_id = od.order_id
),
sub2 AS (
    SELECT 
        p.product_id,
        c.category_name
    FROM 
        products p, categories c
    WHERE 
    p.category_id = c.category_id
)

SELECT
    STRFTIME('%Y', sub1.order_date) year,
    STRFTIME('%m', sub1.order_date) month,
    sub2.category_name,
    ROUND(AVG(sub1.quantity),1) average_unit_saled
FROM 
    sub1, sub2
WHERE 
    sub1.product_id = sub2.product_id
GROUP BY
    1,2,3
ORDER BY
    1,2,4 DESC
"""

df = pd.read_sql_query(query, connection)
df

,year,month,category_name,average_unit_saled
0,2016,01,Electric Bikes,1.7
1,2016,01,Cyclocross Bicycles,1.6
2,2016,01,Comfort Bicycles,1.6
3,2016,01,Cruisers Bicycles,1.5
4,2016,01,Children Bicycles,1.5
...,...,...,...,...
202,2018,11,Road Bikes,1.5
203,2018,11,Mountain Bikes,1.0
204,2018,12,Electric Bikes,2.0
205,2018,12,Road Bikes,1.0


**Question 4** What is the most popular product category in each stores, and how many units have been sold of the top-selling product in that category?

In [49]:
query = """
WITH sub1 AS (
    SELECT
        odi.order_id,
        odi.product_id,
        odi.quantity,
        cat.category_id,
        cat.category_name
    FROM 
        order_items odi 
    JOIN 
        products pd
    ON 
        odi.product_id = pd.product_id
    JOIN 
        categories cat
    ON 
        pd.category_id = cat.category_id
),

sub2 AS (
    SELECT
        od.order_id,
        od.store_id,
        st.store_name
    FROM
        orders od
    JOIN
        stores st
    ON
        od.store_id = st.store_id
),
sub3 AS(
    SELECT
        sub2.store_name,
        sub1.category_name,
        SUM(quantity) total_sales,
        RANK() OVER(PARTITION BY sub2.store_name ORDER BY SUM(quantity) DESC) as rank
    FROM
        sub1
    JOIN
        sub2
    ON
        sub1.order_id = sub2.order_id
    GROUP BY 
        1,2
    ORDER BY 
        1,3 DESC
)

SELECT
    store_name,
    category_name,
    total_sales
FROM 
    sub3
WHERE
    rank = 1
ORDER BY 
    3 DESC;

"""

df = pd.read_sql_query(query, connection)
df

,store_name,category_name,total_sales
0,Baldwin Bikes,Cruisers Bicycles,1396
1,Santa Cruz Bikes,Cruisers Bicycles,422
2,Rowlett Bikes,Cruisers Bicycles,245


**Question 5** What is the average time it takes for an order to be shipped from the time it is placed of each store?

In [50]:
query = """
SELECT
    st.store_id,
    st.store_name,
    ROUND(AVG(julianday(od.shipped_date)- julianday(od.order_date)),2) avg_leadtime
FROM
    orders od, stores st
WHERE 
    od.store_id = st.store_id
GROUP BY
    st.store_id;

"""

df = pd.read_sql_query(query, connection)
df

,store_id,store_name,avg_leadtime
0,1,Santa Cruz Bikes,2.05
1,2,Baldwin Bikes,1.97
2,3,Rowlett Bikes,1.92


**Question 6** Who bought and did not buy in 2018 ?

**Find the customers who brought.**

In [51]:
query = """
SELECT DISTINCT
    od.customer_id,
    cs.first_name || ' ' || cs.last_name AS customer_name 
FROM
    customers cs
JOIN 
    orders od
ON
    od.customer_id = cs.customer_id
WHERE
    CAST(STRFTIME('%Y', od.order_date) AS INT) = 2018
ORDER BY 1 ;

"""

df = pd.read_sql_query(query, connection)
df

,customer_id,customer_name
0,1,Debra Burks
1,2,Kasha Todd
2,3,Tameka Fisher
3,4,Daryl Spence
4,5,Charolette Rice
...,...,...
265,1411,Shanda Stevenson
266,1419,Lurlene Cotton
267,1433,Nicola Knight
268,1441,Jamaal Morrison


**Find the customers who did not brought.**

In [52]:
query = """
WITH sub1 AS (SELECT DISTINCT
    od.customer_id,
    cs.first_name || ' ' || cs.last_name AS customer_name 
FROM
    customers cs
JOIN 
    orders od
ON
    od.customer_id = cs.customer_id
WHERE
    CAST(STRFTIME('%Y', od.order_date) AS INT) = 2018
)

SELECT 
    cs.customer_id,
    cs.first_name || ' ' || cs.last_name AS customer_name 

FROM
    customers cs
LEFT JOIN 
    sub1
ON
    cs.customer_id = sub1.customer_id
WHERE 
    sub1.customer_id IS NULL;




"""

df = pd.read_sql_query(query, connection)
df

,customer_id,customer_name
0,95,Letitia Franco
1,96,Floretta Higgins
2,98,Violet Valenzuela
3,99,Carie Kidd
4,101,Nichelle Howell
...,...,...
1170,1439,Florrie Little
1171,1440,Ernest Rollins
1172,1442,Cassie Cline
1173,1443,Lezlie Lamb


**Question 7** Which the highest pair of products are co-purchased ? 

In [53]:
query = """
SELECT
    pd1.product_name product_a,
    pd2.product_name product_b,
    COUNT(*) count
    
FROM 
    order_items od1,
    order_items od2,
    products pd1,
    products pd2
WHERE 
    od1.order_id = od2.order_id AND
    od1.product_id != od2.product_id AND
    od1.product_id = pd1.product_id AND
    od2.product_id = pd2.product_id
GROUP BY 
    pd1.product_id, pd2.product_id
ORDER BY
    3 DESC ;

"""

df = pd.read_sql_query(query, connection)
df

,product_a,product_b,count
0,Heller Shagamaw Frame - 2016,Electra Girl's Hawaii 1 (16-inch) - 2015/2016,15
1,Electra Girl's Hawaii 1 (16-inch) - 2015/2016,Heller Shagamaw Frame - 2016,15
2,Trek Conduit+ - 2016,Surly Straggler 650b - 2016,14
3,Surly Straggler 650b - 2016,Trek Conduit+ - 2016,14
4,Electra Townie Original 21D - 2016,Electra Cruiser 1 (24-Inch) - 2016,14
...,...,...,...
6923,Electra Townie Original 21D EQ Ladies' - 2018,Trek X-Caliber Frameset - 2018,1
6924,Electra Townie Original 21D Ladies' - 2018,Trek Girl's Kickster - 2017,1
6925,Electra Townie Original 21D Ladies' - 2018,Trek Domane SL 8 Disc - 2018,1
6926,Electra Townie Original 21D Ladies' - 2018,Trek Conduit+ - 2018,1


**Question 8** Are there any customer segments with distinct buying behaviors or preferences?

In [54]:
query = """
WITH sub1 AS (
    SELECT
        o.customer_id,
        SUM(od.quantity * (od.list_price * (1 - od.discount))) AS total_spend,
        COUNT(DISTINCT o.order_id ) total_order
    FROM 
        orders o
    LEFT JOIN
        order_items od
    ON 
        o.order_id = od.order_id
    GROUP BY 
        1
)

SELECT
    customer_id,
    CASE
        WHEN total_spend/(SELECT MAX(total_spend) FROM sub1) >= 0.7 THEN 'High spender'
        WHEN total_spend/(SELECT MAX(total_spend) FROM sub1) <= 0.3 THEN 'Low spender'
        ELSE 'Mid spender' END spender_rank,
    CASE
        WHEN total_order > 1 THEN 'Repeat buyer'
        ELSE 'One-time buyer' END purchase_frequency

FROM sub1

;

"""

df = pd.read_sql_query(query, connection)
df

,customer_id,spender_rank,purchase_frequency
0,1,High spender,Repeat buyer
1,2,Mid spender,Repeat buyer
2,3,Mid spender,Repeat buyer
3,4,Mid spender,Repeat buyer
4,5,Mid spender,Repeat buyer
...,...,...,...
1440,1441,Low spender,One-time buyer
1441,1442,Low spender,One-time buyer
1442,1443,Low spender,One-time buyer
1443,1444,Low spender,One-time buyer


**Question 9** Where are most customers located and can deliveries be made on time?

In [55]:
query = """
SELECT
    city,
    COUNT(*) total_order,
    ROUND(AVG(julianday(o.required_date)-julianday(o.shipped_date)),0) on_time_day,
    CASE
        WHEN AVG(julianday(o.required_date)-julianday(o.shipped_date)) < 0 THEN 'Delay'
        ELSE 'Normal'
        END shipment_status
FROM
    orders o
JOIN
    customers c
ON o.customer_id = c.customer_id
WHERE 
    o.shipped_date IS NOT NULL
GROUP BY
    1
ORDER BY
    2 DESC

;

"""

df = pd.read_sql_query(query, connection)
df

,city,total_order,on_time_day,shipment_status
0,Mount Vernon,20,0.0,Delay
1,Scarsdale,17,0.0,Delay
2,Ballston Spa,17,0.0,Normal
3,Canandaigua,14,0.0,Delay
4,Ossining,13,0.0,Delay
...,...,...,...,...
190,Far Rockaway,2,1.0,Normal
191,Yuba City,1,1.0,Normal
192,Westbury,1,-2.0,Delay
193,Tonawanda,1,-2.0,Delay


**Question 10** Who provides the most service to customers? Who is their supervisor?

In [56]:
query = """
SELECT
    st.store_name,
    s1.staff_id,
    s1.first_name || ' ' || s1.last_name staff_name,
    COUNT(*) service_times,
    s2.first_name || ' ' || s2.last_name manager_name
FROM 
    staffs s1
JOIN
    orders o
ON
    o.staff_id = s1.staff_id
JOIN
    staffs s2
ON
    s1.manager_id = s2.staff_id
JOIN 
    stores st
ON 
    o.store_id = st.store_id
GROUP BY 
    2
ORDER BY 
    3 DESC



;

"""

df = pd.read_sql_query(query, connection)
df

,store_name,staff_id,staff_name,service_times,manager_name
0,Baldwin Bikes,7,Venita Daniel,540,Jannette David
1,Santa Cruz Bikes,2,Mireya Copeland,164,Fabiola Jackson
2,Baldwin Bikes,6,Marcelene Boyer,553,Jannette David
3,Rowlett Bikes,9,Layla Terrell,86,Venita Daniel
4,Rowlett Bikes,8,Kali Vargas,88,Fabiola Jackson
5,Santa Cruz Bikes,3,Genna Serrano,184,Mireya Copeland
